In [10]:
import numpy as np
import pandas as pd

In [1]:
import torch
import torch.optim as optim
from sklearn import preprocessing
# For reproducibility
torch.manual_seed(1)

In [12]:
# df = pd.read_csv("/kaggle/input/2021-ai-w3-p1/train.csv")
train_df = pd.read_csv("./자동차가격예측_data/train_data.csv")
test_df = pd.read_csv("./자동차가격예측_data/test_data.csv")
submit_df = pd.read_csv("./자동차가격예측_data/sample_submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   ID  company    model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2    Focus  2019             1     3764         4  145  34.5   
1   1        1      I10  2015             1    27056         4   20  60.1   
2   2        3       A3  2019             3     3375         4  145  34.5   
3   3        3       A6  2019             0     5743         0  145  47.9   
4   4        2   Fiesta  2015             1    32782         4    0  65.7   

   engineSize  price  
0         2.3  26299  
1         1.0   5495  
2         2.0  32400  
3         2.0  29990  
4         1.0   7250  
   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2        Ka+  2017             1    10150         4  145  57.7   
1   1        2       Kuga  2013             1    50216         0  145  53.3   
2   2        3         Q5  2016             1    34189         0  200  47.9   
3   3        1        I10  2020             1     1900         4  145  56.5   
4   

In [13]:
## preprocessing LabelEncoding "Model" Column
le = preprocessing.LabelEncoder()
le.fit(train_df['model'])
train_df['model'] = le.transform(train_df['model'])
test_df['model'] = le.transform(test_df['model'])
print(le.classes_)

[' 1 Series' ' 2 Series' ' 3 Series' ' 4 Series' ' 5 Series' ' 6 Series'
 ' 7 Series' ' 8 Series' ' A1' ' A3' ' A4' ' A5' ' A6' ' A7' ' A8'
 ' Auris' ' Avensis' ' Aygo' ' B-MAX' ' C-HR' ' C-MAX' ' Camry' ' Corolla'
 ' EcoSport' ' Edge' ' Fiesta' ' Focus' ' Fusion' ' GT86' ' Galaxy'
 ' Getz' ' Grand C-MAX' ' Grand Tourneo Connect' ' Hilux' ' I10' ' I20'
 ' I30' ' I40' ' I800' ' IQ' ' IX20' ' IX35' ' Ioniq' ' KA' ' Ka+' ' Kona'
 ' Kuga' ' Land Cruiser' ' M2' ' M3' ' M4' ' M5' ' M6' ' Mondeo'
 ' Mustang' ' PROACE VERSO' ' Prius' ' Puma' ' Q2' ' Q3' ' Q5' ' Q7' ' Q8'
 ' R8' ' RAV4' ' RS3' ' RS4' ' RS5' ' RS6' ' S-MAX' ' S3' ' S4' ' S5'
 ' S8' ' SQ5' ' SQ7' ' Santa Fe' ' Supra' ' TT' ' Tourneo Connect'
 ' Tourneo Custom' ' Tucson' ' Urban Cruiser' ' Veloster' ' Verso'
 ' Verso-S' ' X1' ' X2' ' X3' ' X4' ' X5' ' X6' ' X7' ' Yaris' ' Z3' ' Z4'
 ' i3' ' i8']


In [14]:
x1_train = torch.FloatTensor(train_df['company']).cuda()
x2_train = torch.FloatTensor(train_df['model']).cuda()
x3_train = torch.FloatTensor(train_df['year']).cuda()
x4_train = torch.FloatTensor(train_df['transmission']).cuda()
x5_train = torch.FloatTensor(train_df['mileage']).cuda()
x6_train = torch.FloatTensor(train_df['fueltype']).cuda()
x7_train = torch.FloatTensor(train_df['tax']).cuda()
x8_train = torch.FloatTensor(train_df['mpg']).cuda()
x9_train = torch.FloatTensor(train_df['engineSize']).cuda()
y_train = torch.FloatTensor(train_df['price']).cuda()

In [15]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True, device = "cuda")
w2 = torch.zeros(1, requires_grad=True,device = "cuda")
w3 = torch.zeros(1, requires_grad=True,device = "cuda")
w4 = torch.zeros(1,requires_grad= True,device = "cuda")
w5 = torch.zeros(1, requires_grad=True,device = "cuda")
w6 = torch.zeros(1, requires_grad=True,device = "cuda")
w7 = torch.zeros(1, requires_grad=True,device = "cuda")
w8 = torch.zeros(1,requires_grad= True,device = "cuda")
w9 = torch.zeros(1,requires_grad= True,device = "cuda")
b = torch.zeros(1, requires_grad=True,device = "cuda")
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3,w4,w5,w6,w7,w8,w9, b], lr=2e-10)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + x4_train * w4 + x5_train * w5+ x6_train * w6+x7_train * w7+x8_train * w8+x9_train * w9 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000  Cost: 383204160.000000
Epoch 1000/10000  Cost: 130353784.000000
Epoch 2000/10000  Cost: 93884312.000000
Epoch 3000/10000  Cost: 85415368.000000
Epoch 4000/10000  Cost: 83435952.000000
Epoch 5000/10000  Cost: 82960560.000000
Epoch 6000/10000  Cost: 82833784.000000
Epoch 7000/10000  Cost: 82787864.000000
Epoch 8000/10000  Cost: 82760744.000000
Epoch 9000/10000  Cost: 82738040.000000
Epoch 10000/10000  Cost: 82716384.000000


In [16]:
x1_test = torch.FloatTensor(test_df['company']).cuda()
x2_test = torch.FloatTensor(test_df['model']).cuda()
x3_test = torch.FloatTensor(test_df['year']).cuda()
x4_test = torch.FloatTensor(test_df['transmission']).cuda()
x5_test = torch.FloatTensor(test_df['mileage']).cuda()
x6_test = torch.FloatTensor(test_df['fueltype']).cuda()
x7_test = torch.FloatTensor(test_df['tax']).cuda()
x8_test = torch.FloatTensor(test_df['mpg']).cuda()
x9_test = torch.FloatTensor(test_df['engineSize']).cuda()
y_test = x1_test * w1 + x2_test * w2 + x3_test * w3 + x4_test * w4 + x5_test * w5+ x6_test * w6+x7_test * w7+x8_test * w8+x9_test * w9 + b
print(y_test)

tensor([19474.5098, 11760.6221, 14937.2871,  ..., 12463.4561, 18619.0195,
         6215.1138], device='cuda:0', grad_fn=<AddBackward0>)


In [17]:
submit_df['price'] = y_test.cpu().detach().numpy()
submit_df.head()

,ID,price
0,0,19474.509766
1,1,11760.622070
2,2,14937.287109
3,3,21082.390625
4,4,5927.687012


In [18]:
submit_df.to_csv("./배추가격예측_data/submit_sample.csv",index=False)